### Import data

In [148]:
import pandas as pd
import numpy as np
import re

In [149]:
# cnbc = pd.read_csv(r'\Users\39118\Desktop\9665\NLPdata\cnbc_headlines.csv')
# guar = pd.read_csv(r'\Users\39118\Desktop\9665\NLPdata\guardian_headlines.csv')
# reut = pd.read_csv(r'\Users\39118\Desktop\9665\NLPdata\reuters_headlines.csv')
# data = pd.concat([cnbc, guar, reut], axis = 0)
# data.to_csv('data_combined.csv')
# dataT = data.copy()
# dataL = dataT[['Headlines','Time']].dropna()
# dataL.shape
# dataL['Year'] = [re.split('[-\s]', var)[-1] for var in dataL['Time']]
# dataL['Year'] = ['20'+ item if len(item) == 2 else item for item in dataL['Year']]
# dataL['Year'] = pd.to_datetime(dataL['Year'], format = '%Y')
# dataL = dataL.set_index('Year')
# dataL.to_excel('News Dataset for Project.xlsx')
data = pd.read_excel('News Dataset for Project.xlsx')

In [150]:
news_data = data.copy()

In [151]:
headlines_data = news_data[['Headlines']]

### Data Preprocessing

In [152]:
from nltk.corpus import stopwords 
import nltk

In [153]:
from nltk.corpus import stopwords 
import nltk

In [154]:
words = stopwords.words('english')

In [155]:
# build customized stopwords
stopwords = ['i',"'s'",'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 
             "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 
             'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 
             'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 
             'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 
             'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 
             'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 
             'a', 'an', 'the', 'and', 'if', 'or', 'because', 'as', 'until', 'while', 
             'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through',
             'during', 'before', 'after', 'to', 'from', 'in', 'out', 'on', 'over', 'under', 
             'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 
             'how', 'all', 'any', 'both', 'each', 'more', 'most', 'other', 'some', 'such',
             'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren']
negations = words[-35:].extend(['don',"don't",'few','up','down','under','above','against','off','not','no','nor'])
punctuation = ['.','?',";",'...','"','()','\'',',','_','!','--']
special_punc = [':']
high_freq_useless = ['jim','cramer','said','us','ceo','say','says']

In [156]:

def preprocess_news_data(data,name):
    data[name] = data[name].str.lower()
#     data[name] = data[name].apply(lambda x:re.sub(r'','not',x))
preprocess_news_data(headlines_data,'Headlines')

<ipython-input-156-b1157b4f5183>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[name] = data[name].str.lower()


In [157]:
# Stemmer initialization
wnl = nltk.WordNetLemmatizer()

In [158]:
# tokenize new headlines
headlines_data['Tokenized_Headlines'] = [nltk.word_tokenize(sent) 
                                         for sent in headlines_data.Headlines]

<ipython-input-158-29544a1c44fa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  headlines_data['Tokenized_Headlines'] = [nltk.word_tokenize(sent)


In [159]:
Tokenized_Headlines = []
for i in headlines_data['Tokenized_Headlines']:
    Tokenized_Headlines.append([word.strip("'") for word in i])
headlines_data['Tokenized_Headlines'] = Tokenized_Headlines

<ipython-input-159-0177349d7759>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  headlines_data['Tokenized_Headlines'] = Tokenized_Headlines


In [160]:
# replace n't with not
for i in headlines_data.Tokenized_Headlines:
    for j in range(len(i)):
        if i[j] == "n't" :
            i[j] = 'not'

In [161]:
def normal_tokenized(tokens) -> list:
    '''normalize,stem and remove stopwords'''
    Normal_Tokenized_Headlines = []
    for token in tokens:
        Normal_Tokenized_Headlines.append([wnl.lemmatize(word).lower()
                                           for word in token 
                                           if word.lower() not in stopwords and
                                           word.isalpha() or word in special_punc]) 
    return Normal_Tokenized_Headlines

In [162]:
headlines_data['Normal_Tokenized_Headlines'] = normal_tokenized(headlines_data['Tokenized_Headlines'])

In [163]:
def index(tokens) -> list:
    '''remove irrelevant content and text before semicolon'''
    index_colon_apply = []
    index_say_apply = []
    for i in tokens:
        if ':' in i:
            index_colon_apply.append(i.index(':'))
        else:
            index_colon_apply.append(0)
        if 'say' in i:
            index_say_apply.append(i.index('say'))
        else:
            index_say_apply.append(0)
           
    return index_colon_apply, index_say_apply

In [164]:
# create the index for the point where may contain the useless information
index_colon_apply, index_say_apply = index(headlines_data['Normal_Tokenized_Headlines'])
headlines_data['index_colon_apply'] = index_colon_apply
headlines_data['index_say_apply'] = index_say_apply
headlines_data['tokens_len'] = [len(token)-1 for token in headlines_data['Normal_Tokenized_Headlines']]

In [165]:
headlines_data.head()

,Headlines,Tokenized_Headlines,Normal_Tokenized_Headlines,index_colon_apply,index_say_apply,tokens_len
0,jim cramer: a better way to invest in the covi...,"[jim, cramer, :, a, better, way, to, invest, i...","[jim, cramer, :, better, way, invest, vaccine,...",2,0,8
1,cramer's lightning round: i would own teradyne,"[cramer, s, lightning, round, :, i, would, own...","[cramer, lightning, round, :, would, teradyne]",3,0,5
2,"cramer's week ahead: big week for earnings, ev...","[cramer, s, week, ahead, :, big, week, for, ea...","[cramer, week, ahead, :, big, week, earnings, ...",3,0,10
3,iq capital ceo keith bliss says tech and healt...,"[iq, capital, ceo, keith, bliss, says, tech, a...","[iq, capital, ceo, keith, bliss, say, tech, he...",0,5,8
4,wall street delivered the 'kind of pullback i'...,"[wall, street, delivered, the, kind, of, pullb...","[wall, street, delivered, kind, pullback, wait...",0,8,8


In [166]:
def remove_colon_say(news, index1, index3):
    Headlines_without_colon = []
    Headlines_without_say = []
    for i in range(len(news)):
        if index3[i] - index1[i] <= 1:
            Headlines_without_colon.append(news[i][:index1[i]])
        else:
            Headlines_without_colon.append(news[i][index1[i]+1:])
            
    return Headlines_without_colon

In [167]:
Headlines_without_colon = remove_colon_say(headlines_data['Normal_Tokenized_Headlines'],  headlines_data['index_colon_apply']
                                        ,headlines_data['tokens_len'])

In [168]:
headlines_data['Headlines_without_colon'] = Headlines_without_colon

In [169]:
withou_colon_len = []
for i in headlines_data.Headlines_without_colon:
    withou_colon_len.append(len(i))

In [170]:
headlines_data['withou_colon_len'] = withou_colon_len

In [171]:
without_colon = headlines_data.Headlines_without_colon.tolist()
tokens =  headlines_data.tokens_len.tolist()
len_diff = []
for i in range(len(without_colon)):
    len_diff.append(tokens[i]-len(without_colon[i]))

In [172]:
headlines_data['len_diff'] = len_diff

In [173]:
headlines_data

,Headlines,Tokenized_Headlines,Normal_Tokenized_Headlines,index_colon_apply,index_say_apply,tokens_len,Headlines_without_colon,withou_colon_len,len_diff
0,jim cramer: a better way to invest in the covi...,"[jim, cramer, :, a, better, way, to, invest, i...","[jim, cramer, :, better, way, invest, vaccine,...",2,0,8,"[better, way, invest, vaccine, gold, rush]",6,2
1,cramer's lightning round: i would own teradyne,"[cramer, s, lightning, round, :, i, would, own...","[cramer, lightning, round, :, would, teradyne]",3,0,5,"[would, teradyne]",2,3
2,"cramer's week ahead: big week for earnings, ev...","[cramer, s, week, ahead, :, big, week, for, ea...","[cramer, week, ahead, :, big, week, earnings, ...",3,0,10,"[big, week, earnings, even, bigger, week, vacc...",7,3
3,iq capital ceo keith bliss says tech and healt...,"[iq, capital, ceo, keith, bliss, says, tech, a...","[iq, capital, ceo, keith, bliss, say, tech, he...",0,5,8,"[capital, ceo, keith, bliss, say, tech, health...",8,0
4,wall street delivered the 'kind of pullback i'...,"[wall, street, delivered, the, kind, of, pullb...","[wall, street, delivered, kind, pullback, wait...",0,8,8,"[street, delivered, kind, pullback, waiting, j...",8,0
...,...,...,...,...,...,...,...,...,...
53365,malaysia says never hired british data firm at...,"[malaysia, says, never, hired, british, data, ...","[malaysia, say, never, hired, british, data, f...",0,1,8,"[say, never, hired, british, data, firm, cente...",8,0
53366,prosecutors search volkswagen headquarters in ...,"[prosecutors, search, volkswagen, headquarters...","[prosecutor, search, volkswagen, headquarters,...",0,0,6,"[search, volkswagen, headquarters, new, emissi...",6,0
53367,mcdonald's sets greenhouse gas reduction targets,"[mcdonald, s, sets, greenhouse, gas, reduction...","[mcdonald, set, greenhouse, gas, reduction, ta...",0,0,5,"[set, greenhouse, gas, reduction, target]",5,0
53368,pratt & whitney to deliver spare a320neo engin...,"[pratt, &, whitney, to, deliver, spare, a320ne...","[pratt, whitney, deliver, spare, engine, soon,...",8,0,9,"[pratt, whitney, deliver, spare, engine, soon,...",8,1


In [174]:
len_diff = headlines_data['len_diff'].tolist()
for i in range(len(len_diff)):
    if len_diff[i] >= 5:
         headlines_data.Headlines_without_colon[i] = headlines_data.Normal_Tokenized_Headlines[i]

<ipython-input-174-bfeff03dceee>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  headlines_data.Headlines_without_colon[i] = headlines_data.Normal_Tokenized_Headlines[i]


In [175]:
def normal_tokenized_further(tokens) -> list:
    '''further data cleanning'''
    Tokens = []
    for token in tokens:
        Tokens.append([wnl.lemmatize(word).lower()
                                           for word in token 
                                           if word.lower() not in stopwords and
                                           word.isalpha() and word not in special_punc
                                           and word not in high_freq_useless]) 
    return Tokens

In [176]:
Tokens = normal_tokenized_further(headlines_data.Headlines_without_colon)

In [177]:
headlines_data['Tokens'] = Tokens

In [178]:
# freq = nltk.FreqDist(word for sent in headlines_data.Headlines_without_colon for word in sent)
# freq.most_common()

In [179]:
new_headlines_data = headlines_data.loc[:,['Tokens']]

In [180]:
new_headlines_data
# use for sentiment Analysis

,Tokens
0,"[better, way, invest, vaccine, gold, rush]"
1,"[would, teradyne]"
2,"[big, week, earnings, even, bigger, week, vacc..."
3,"[capital, keith, bliss, tech, healthcare, rally]"
4,"[street, delivered, kind, pullback, waiting]"
...,...
53365,"[never, hired, british, data, firm, center, sc..."
53366,"[search, volkswagen, headquarters, new, emissi..."
53367,"[set, greenhouse, gas, reduction, target]"
53368,"[pratt, whitney, deliver, spare, engine, soon,..."


In [181]:
import nltk
import ssl
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


In [182]:
lemmatizer = WordNetLemmatizer()

In [183]:
postagging = [nltk.pos_tag(sent) for sent in new_headlines_data['Tokens']]

In [184]:
pos=neg=obj=count=0
new_headlines_data['pos_tag'] = postagging

def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None



def get_sentiment(word,tag):
    wn_tag = penn_to_wn(tag)
    
    if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
        return []

    #Lemmatization
    lemma = lemmatizer.lemmatize(word, pos=wn_tag)
    if not lemma:
        return []

    #Synset is a special kind of a simple interface that is present in NLTK to look up words in WordNet. 
    #Synset instances are the groupings of synonymous words that express the same concept. 
    #Some of the words have only one Synset and some have several.
    synsets = wn.synsets(word, pos=wn_tag)
    if not synsets:
        return []

    # Take the first sense, the most common
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name())

    return [synset.name(), swn_synset.pos_score(),swn_synset.neg_score(),swn_synset.obj_score()]

    pos=neg=obj=count=0
    
    ###################################################################################
senti_score = []

for pos_val in new_headlines_data['pos_tag']:
    senti_val = [get_sentiment(x,y) for (x,y) in pos_val]
    for score in senti_val:
        try:
            pos = pos + score[1]  #positive score is stored at 2nd position
            neg = neg + score[2]  #negative score is stored at 3rd position
        except:
            continue
    senti_score.append(pos - neg)
    pos=neg=0    
    
new_headlines_data['senti_score'] = senti_score
print(new_headlines_data['senti_score'])

print(new_headlines_data.head)

0        0.875
1        0.000
2        0.375
3        0.250
4        0.000
         ...  
53365   -0.625
53366    0.375
53367    0.000
53368    0.125
53369    0.000
Name: senti_score, Length: 53370, dtype: float64
<bound method NDFrame.head of                                                   Tokens  \
0             [better, way, invest, vaccine, gold, rush]   
1                                      [would, teradyne]   
2      [big, week, earnings, even, bigger, week, vacc...   
3       [capital, keith, bliss, tech, healthcare, rally]   
4           [street, delivered, kind, pullback, waiting]   
...                                                  ...   
53365  [never, hired, british, data, firm, center, sc...   
53366  [search, volkswagen, headquarters, new, emissi...   
53367          [set, greenhouse, gas, reduction, target]   
53368  [pratt, whitney, deliver, spare, engine, soon,...   
53369  [uk, always, consider, way, improve, data, law...   

                                   

In [185]:
overall=[]
for i in range(len(new_headlines_data)):
    if new_headlines_data['senti_score'][i]> 0:
        overall.append('Positive')
    elif new_headlines_data['senti_score'][i]< 0:
        overall.append('Negative')
    else:
        overall.append('Neutral')
new_headlines_data['Overall Sentiment']=overall

In [186]:
new_headlines_data

,Tokens,pos_tag,senti_score,Overall Sentiment
0,"[better, way, invest, vaccine, gold, rush]","[(better, RBR), (way, NN), (invest, JJ), (vacc...",0.875,Positive
1,"[would, teradyne]","[(would, MD), (teradyne, VB)]",0.000,Neutral
2,"[big, week, earnings, even, bigger, week, vacc...","[(big, JJ), (week, NN), (earnings, NNS), (even...",0.375,Positive
3,"[capital, keith, bliss, tech, healthcare, rally]","[(capital, NN), (keith, NN), (bliss, JJ), (tec...",0.250,Positive
4,"[street, delivered, kind, pullback, waiting]","[(street, NN), (delivered, VBD), (kind, NN), (...",0.000,Neutral
...,...,...,...,...
53365,"[never, hired, british, data, firm, center, sc...","[(never, RB), (hired, VBN), (british, JJ), (da...",-0.625,Negative
53366,"[search, volkswagen, headquarters, new, emissi...","[(search, NN), (volkswagen, NN), (headquarters...",0.375,Positive
53367,"[set, greenhouse, gas, reduction, target]","[(set, VBN), (greenhouse, NN), (gas, NN), (red...",0.000,Neutral
53368,"[pratt, whitney, deliver, spare, engine, soon,...","[(pratt, NN), (whitney, NN), (deliver, NN), (s...",0.125,Positive


In [187]:
labeled_data = pd.concat([news_data, new_headlines_data], axis = 1)

In [188]:
labeled_data 

,Year,Headlines,Time,Tokens,pos_tag,senti_score,Overall Sentiment
0,2020-01-01,Jim Cramer: A better way to invest in the Covi...,"7:51 PM ET Fri, 17 July 2020","[better, way, invest, vaccine, gold, rush]","[(better, RBR), (way, NN), (invest, JJ), (vacc...",0.875,Positive
1,2020-01-01,Cramer's lightning round: I would own Teradyne,"7:33 PM ET Fri, 17 July 2020","[would, teradyne]","[(would, MD), (teradyne, VB)]",0.000,Neutral
2,2020-01-01,"Cramer's week ahead: Big week for earnings, ev...","7:25 PM ET Fri, 17 July 2020","[big, week, earnings, even, bigger, week, vacc...","[(big, JJ), (week, NN), (earnings, NNS), (even...",0.375,Positive
3,2020-01-01,IQ Capital CEO Keith Bliss says tech and healt...,"4:24 PM ET Fri, 17 July 2020","[capital, keith, bliss, tech, healthcare, rally]","[(capital, NN), (keith, NN), (bliss, JJ), (tec...",0.250,Positive
4,2020-01-01,Wall Street delivered the 'kind of pullback I'...,"7:36 PM ET Thu, 16 July 2020","[street, delivered, kind, pullback, waiting]","[(street, NN), (delivered, VBD), (kind, NN), (...",0.000,Neutral
...,...,...,...,...,...,...,...
53365,2018-01-01,Malaysia says never hired British data firm at...,Mar 20 2018,"[never, hired, british, data, firm, center, sc...","[(never, RB), (hired, VBN), (british, JJ), (da...",-0.625,Negative
53366,2018-01-01,Prosecutors search Volkswagen headquarters in ...,Mar 20 2018,"[search, volkswagen, headquarters, new, emissi...","[(search, NN), (volkswagen, NN), (headquarters...",0.375,Positive
53367,2018-01-01,McDonald's sets greenhouse gas reduction targets,Mar 20 2018,"[set, greenhouse, gas, reduction, target]","[(set, VBN), (greenhouse, NN), (gas, NN), (red...",0.000,Neutral
53368,2018-01-01,Pratt & Whitney to deliver spare A320neo engin...,Mar 20 2018,"[pratt, whitney, deliver, spare, engine, soon,...","[(pratt, NN), (whitney, NN), (deliver, NN), (s...",0.125,Positive


In [189]:
labeled_data.to_excel('labeled_data_11_29.xlsx')